In [1]:
"""
전체 파이프라인 통합 테스트
"""
import sys
sys.path.append('..')
import nest_asyncio
nest_asyncio.apply()


from chains.classifier import classify_question
from chains.rag_chain import query_rag
from chains.stock_chain import query_stock_analysis
from chains.general_chain import query_general_advice
import asyncio
from chains.indicator_chain import query_economic_indicator

async def test_full_pipeline():
    """전체 파이프라인 테스트"""
    
    test_questions = [
        # analyst_report
        ("삼성전자 AI 반도체 전망은?", "analyst_report"),
        
        # stock_price
        ("삼성전자 주가가 얼마야?", "stock_price"),
        
        # economic_indicator
        ("기준금리가 주식시장에 미치는 영향은?", "economic_indicator"),
        
        # general
        ("초보자한테 추천하는 투자 전략은?", "general")
    ]
    
    for question, expected_category in test_questions:
        print(f"\n{'='*70}")
        print(f"질문: {question}")
        print(f"예상 카테고리: {expected_category}")
        print('='*70)
        
        # 1. 질문 분류
        classification = classify_question(question)
        category = classification["category"]
        stock_code = classification.get("stock_code")
        
        print(f"\n[분류] {category} (종목: {stock_code})")
        
        # 2. 카테고리별 처리
        if category == "analyst_report":
            result = query_rag(question)
            print(f"\n[답변]\n{result['answer']}")
            print(f"\n[출처]")
            for src in result['sources']:
                print(f"  - {src}")
        
        elif category == "stock_price":
            if stock_code:
                answer = query_stock_analysis(question, stock_code)
                print(f"\n[답변]\n{answer}")
            else:
                print("\n[답변] 종목 코드를 인식하지 못했습니다.")
        
        elif category == "economic_indicator":
            answer = await query_economic_indicator(question)
            print(f"\n[답변]\n{answer}")
        
        else:  # general
            answer = query_general_advice(question)
            
            print(f"\n[답변]\n{answer}")

if __name__ == "__main__":
    print("전체 파이프라인 테스트 시작\n")
    asyncio.run(test_full_pipeline())
    print("\n테스트 완료!")

c:\Users\user\anaconda3\envs\myenv\lib\site-packages\pykrx\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-10-27 06:28:51,735 - utils.logger - INFO - 질문 분류 시작: 삼성전자 AI 반도체 전망은?


전체 파이프라인 테스트 시작


질문: 삼성전자 AI 반도체 전망은?
예상 카테고리: analyst_report


2025-10-27 06:28:52,749 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_ope************here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}